In [1]:
# 파일 경로 지정
import sys, os
from IPython import get_ipython
notebook_dir = os.path.dirname(os.path.abspath(get_ipython().run_line_magic('pwd', '')))
sys.path.append(notebook_dir)

In [2]:
# 데이터 로드
import pandas as pd
from lib import data_preprocessor

# 데이터 csv를 dataFrame으로 변환
df = data_preprocessor.transform_verilog_results_to_DataFrame('../data/data_206_col.csv', skiprows=0)
normal_df, data_min, data_max = data_preprocessor.normalize_DataFrame(df)

input_feature = ['i' + str(i) + '_high' for i in range(1, 61)] + ['i' + str(i) + '_low' for i in range(1, 61)] + ['c' + str(i) + '_low' for i in range(1, 61)]

output_feature = ['vth0', 'cit', 'mob_u0', 'mob_ua', 'mob_ub', 'mob_ug', 'nfactor', 'delta', 'kg', 'nlx', 'voff', 'cf', 'cgsl', 'cgso', 'cgdo', 'cgdl', 'clc', 'delvt', 'dwc', 'cjswg', 'csdesw', 'dlbg', 'dlc', 'moin']

# 가져온 데이터의 열 이름이 input_feature와 output_feature에 있는지 확인
if ['index'] + output_feature + input_feature == df.columns.tolist():
    print('열 이름이 일치합니다: ', df.columns.tolist())
else:
    raise ValueError('열 이름이 일치하지 않습니다: ', df.columns.tolist())

[] columns are dropped.
열 이름이 일치합니다:  ['index', 'vth0', 'cit', 'mob_u0', 'mob_ua', 'mob_ub', 'mob_ug', 'nfactor', 'delta', 'kg', 'nlx', 'voff', 'cf', 'cgsl', 'cgso', 'cgdo', 'cgdl', 'clc', 'delvt', 'dwc', 'cjswg', 'ckappas', 'csdesw', 'dlbg', 'dlc', 'moin', 'i1_high', 'i2_high', 'i3_high', 'i4_high', 'i5_high', 'i6_high', 'i7_high', 'i8_high', 'i9_high', 'i10_high', 'i11_high', 'i12_high', 'i13_high', 'i14_high', 'i15_high', 'i16_high', 'i17_high', 'i18_high', 'i19_high', 'i20_high', 'i21_high', 'i22_high', 'i23_high', 'i24_high', 'i25_high', 'i26_high', 'i27_high', 'i28_high', 'i29_high', 'i30_high', 'i31_high', 'i32_high', 'i33_high', 'i34_high', 'i35_high', 'i36_high', 'i37_high', 'i38_high', 'i39_high', 'i40_high', 'i41_high', 'i42_high', 'i43_high', 'i44_high', 'i45_high', 'i46_high', 'i47_high', 'i48_high', 'i49_high', 'i50_high', 'i51_high', 'i52_high', 'i53_high', 'i54_high', 'i55_high', 'i56_high', 'i57_high', 'i58_high', 'i59_high', 'i60_high', 'i1_low', 'i2_low', 'i3_low', '

In [3]:
# 모델 로드
import torch, joblib
from torch.utils.data import DataLoader
from lib import ANN
from lib import CNN_new
from lib import LSTM

print(len(input_feature), len(output_feature))

ann_model = ANN.ANN(len(input_feature), len(output_feature))
ann_model.load_state_dict(torch.load('../Model/ann_model_weights.pth'))
ann_model.eval()

rf_model = joblib.load('../Model/random_forest_model.pkl')

cnn_model = CNN_new.CNN(len(input_feature), len(output_feature))
cnn_model.load_state_dict(torch.load('../Model/cnn_model_weights.pth'))
cnn_model.eval()

gbm_model = joblib.load('../Model/gbm_model.pkl')

lstm_model = LSTM.LSTM(len(input_feature), len(output_feature))
lstm_model.load_state_dict(torch.load('../Model/lstm_model_weights.pth'))
lstm_model.eval()

180 25


C:\Users\LG\AppData\Local\Temp\ipykernel_8480\3781276938.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ann_model.load_state_dict(torch.load('../Model/ann_model_weight

LSTM(
  (lstm): LSTM(180, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=25, bias=True)
)

In [29]:
# 모델 input 데이터 생성
import numpy as np
import pickle, json
from lib import SSHManager, app, model_testing

# 데이터 중 랜덤하게 n개를 뽑아 테스트 데이터로 사용한다.
n = 2
test_df = normal_df.sample(n)

# 모델들을 dict로 저장한다.
model_names = ['ann', 'rf', 'cnn', 'gbm', 'lstm']
models = [ann_model, rf_model, cnn_model, gbm_model, lstm_model]

name_pytorch = ['ann', 'cnn', 'lstm']
name_sklearn = ['rf', 'gbm']

# 각 모델들의 결과를 저장할 dict를 생성한다.
# 결과를 list로 저장되고, 각 list는 model_names를 key로 한다.
result_parameters: dict = {name: [] for name in model_names}
result_i: dict = {name: [] for name in model_names}

# 서버에 연결한다.
server_info = json.load(open("../server_info.json", "r"))
ssh = SSHManager.SSHManager(server_info["host"], server_info["port"], server_info["id"], server_info["password"])

number_of_parameter = len(output_feature)
i_max = data_max[input_feature].to_numpy()
i_min = data_min[input_feature].to_numpy()

for model, name in zip(models, model_names):

    print(f"\n\nModel: {name}\n\n")

    # i_ons, i_offs, HSPICE_i를 초기화한다.
    parameters = []
    HSPICE_output: list = []
    index = 0

    for _, data in test_df.iterrows():

        # input_feature에 대한 데이터를 추출한다.
        normal_i = data[input_feature].to_numpy()
        unnormal_i = normal_i * (data_max[input_feature] - data_min[input_feature]) + data_min[input_feature]


        # model이 사용된 라이브러리에 따라 다른 input을 사용한다.
        if name in name_pytorch:
            pred_pytorch = model(torch.tensor(normal_i, dtype=torch.float32).unsqueeze(0))
            pred_pytorch_numpy = pred_pytorch.detach().numpy().squeeze()

            # 결과를 역정규화한다.
            unnormal_pred = pred_pytorch_numpy * (data_max[output_feature] - data_min[output_feature]) + data_min[output_feature]

        # 만약, model이 sklearn 모델이라면,
        elif name in name_sklearn:
            df = pd.DataFrame([normal_i], columns=input_feature)
            pred_sklearn = model.predict(df)

            # 결과를 역정규화한다.
            # rf는 8개의 데이터를 가진 numpy array를 묶어서 반환하기 때문에 [0]을 붙여준다.
            unnormal_pred = pred_sklearn[0] * (data_max[output_feature] - data_min[output_feature]) + data_min[output_feature]
            

        # 모두 해당되지 않는다면, 에러를 출력한다.
        else:
            raise ValueError("Model name error")

        # unnormal_pred를 저장한다.
        parameters.append(unnormal_pred)

        # HSPICE를 통해 결과를 받아온다.
        data_from_server = model_testing.get_test_data_from_server(ssh, ['sp_and_nmos/TFT_BSIM_high.sp', 'sp_and_nmos/TFT_BSIM_low.sp'], 'sp_and_nmos/BSIM_tft_cv.pmos', input_feature, unnormal_pred, 800)
        HSPICE_output.append(data_from_server)

        # 만약 index가 0이라면, 출력한다.
        if index == 0:
            print(f'model: {model}')
            print(f'parameters: {unnormal_pred}')
            print(f"HSPICE_output: {data_from_server}")
        if index % 10 == 0:
            print(f'{name}의 {index}번째 실행 중...')
        index += 1

    print(f'{name}이 종료됨. 실행 길이: {len(HSPICE_output), len(parameters)}')

    # i_on, i_off에 대한 for문이 종료되면 결과를 저장한다.
    result_parameters[name].append(parameters)
    result_i[name].append(HSPICE_output)

# 서버와의 연결을 종료한다.
ssh.close()



Model: ann


model: ANN(
  (fc1): Linear(in_features=180, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=1000, bias=True)
  (fc4): Linear(in_features=1000, out_features=1000, bias=True)
  (fc5): Linear(in_features=1000, out_features=1000, bias=True)
  (fc6): Linear(in_features=1000, out_features=1000, bias=True)
  (fc7): Linear(in_features=1000, out_features=25, bias=True)
)
parameters: vth0      -5.003539e-01
cit        5.909329e-03
mob_u0     5.011887e+03
mob_ua     7.977412e-12
mob_ub     1.002605e-18
mob_ug     2.338895e-03
nfactor    3.420616e+00
delta      1.482860e+00
kg         7.432942e-01
nlx        4.983151e-06
voff      -1.301282e+00
cf         4.980179e-20
cgsl       8.997978e-20
cgso       2.670389e-10
cgdo       1.025155e-10
cgdl       5.002655e-12
clc        3.994114e-10
delvt      3.237805e+00
dwc        9.253822e-07
cjswg      5.020625e-05
ckappas    4.981312e-01
csdesw    

In [41]:
# 결과를 csv로 저장한다.
# 각 모델 별로 dataFrame을 만들고 합쳐 csv로 저장한다.
results_df = {name: pd.DataFrame() for name in model_names}
for name in model_names:

    print(f"\n\n{name} 결과 저장 중...\n\n")

    # [name]으로 불러온 df의 column의 이름을 원래 column의 이름에 _{name}을 붙여서 저장한다.
    parameters = np.array(result_parameters[name]).squeeze(0)
    parameters = pd.DataFrame(parameters, columns=[feature + f"_{name}" for feature in output_feature])

    output = np.array(result_i[name]).squeeze()
    output = pd.DataFrame(output, columns=[feature + f"_{name}" for feature in input_feature])

    # 둘을 합친다.
    result_df = pd.concat([parameters, output], axis=1)
    results_df[name] = result_df

    # output, parameter의 첫번째 값을 출력한다.
    print(f"output: {output.iloc[0]}")
    print(f"parameter: {parameters.iloc[0]}")

# i_original_df를 index가 제거된 상태로 저장한다.
i_original_df = test_df[input_feature] * (data_max[input_feature] - data_min[input_feature]) + data_min[input_feature]
i_original_df = i_original_df.reset_index(drop=True)

# result_df를 합친다.
result_list = [i_original_df] + [results_df[name] for name in model_names]
result_df = pd.concat(result_list, axis=1)

# result_df를 csv로 저장한다.
result_df.to_csv('result.csv', index=False)

# 엑셀로 저장한다.
result_df.to_excel('result.xlsx', index=False)



ann 결과 저장 중...


output: i1_high_ann   -1.894800e-06
i2_high_ann   -1.803100e-06
i3_high_ann   -1.712400e-06
i4_high_ann   -1.622800e-06
i5_high_ann   -1.534400e-06
                   ...     
c56_low_ann    5.507615e-16
c57_low_ann    5.479965e-16
c58_low_ann    5.451297e-16
c59_low_ann    5.367900e-16
c60_low_ann    5.336781e-16
Name: 0, Length: 180, dtype: float64
parameter: vth0_ann      -5.003539e-01
cit_ann        5.909329e-03
mob_u0_ann     5.011887e+03
mob_ua_ann     7.977412e-12
mob_ub_ann     1.002605e-18
mob_ug_ann     2.338895e-03
nfactor_ann    3.420616e+00
delta_ann      1.482860e+00
kg_ann         7.432942e-01
nlx_ann        4.983151e-06
voff_ann      -1.301282e+00
cf_ann         4.980179e-20
cgsl_ann       8.997978e-20
cgso_ann       2.670389e-10
cgdo_ann       1.025155e-10
cgdl_ann       5.002655e-12
clc_ann        3.994114e-10
delvt_ann      3.237805e+00
dwc_ann        9.253822e-07
cjswg_ann      5.020625e-05
ckappas_ann    4.981312e-01
csdesw_ann     1.996337e-05
d

In [ ]:
# 결과를 matplotlib으로 출력

import matplotlib.pyplot as plt
import pandas as pd, numpy as np

result_df = pd.read_csv('result.csv')
model_names = ['ann', 'rf', 'cnn', 'gbm', 'lstm']

# 출력하고자 하는 대상은, 각 colum에 대한 error의 평균, 가장 큰 error를 가지는 index의 값, 가장 작은 error를 가지는 index의 값, 무작위로 선택된 index의 값이다.
fig: plt.Figure = None
ax: plt.Axes = None
fig, ax = plt.subplots(len(model_names), 4, figsize=(30, 5 * len(model_names)))

for name, index in zip(model_names, range(len(model_names))):
    params = result_df[[column + '_' + name for column in output_feature]].to_numpy()
    i_original = result_df[input_feature].to_numpy()
    output = result_df[[column + '_' + name for column in input_feature]].to_numpy()

    # i0~i62 각각에 대한 error 목록 (퍼센트)
    i_error = np.abs((i_original - output) / output) * 100
    print(f'{name}의 error: {i_error}')

    # 그 평균에 해당하는 값 (퍼센트)
    mean_error = i_error.mean(axis=0)

    # index 별 error
    index_error = i_error.mean(axis=1)

    # mean_error의 합산이 가장 큰 index를 찾아, 그래프 위에 해당 index의 i와 original_i를 모두 표시한다.
    max_mean_error_index = index_error.argmax()

    # mean_error의 합산이 가장 작은 index를 찾아, 그래프 위에 해당 index의 i와 original_i를 모두 표시한다.
    min_mean_error_index = index_error.argmin()
    
    # 무작위로 선택된 하나의 index에 대해, 그래프 위에 해당 index의 i와 original_i를 모두 표시한다.
    random_index = np.random.randint(0, i_error.shape[0])

    # 변수 설정
    tick_spacing = 5

    # 그래프를 그린다.
    x = range(len(input_feature))

    # mean_error 그래프
    ax[index, 0].plot(x, mean_error, label='Mean Error (%)')
    ax[index, 0].set_xticks(range(0, len(input_feature), tick_spacing))
    ax[index, 0].set_xticklabels(input_feature[::5], rotation=90)
    ax[index, 0].set_xlabel('i')
    ax[index, 0].set_ylabel('Mean Error (%)')
    ax[index, 0].set_title(name + '_mean_error')

    # max_i_error 그래프
    ax1 = ax[index, 1]
    ax2 = ax1.twinx()
    ax1.plot(x, i_error[max_mean_error_index], 'r-', label='Error')
    ax2.plot(x, i_original[max_mean_error_index], 'b-', label='Original')
    ax2.plot(x, output[max_mean_error_index], 'g-', label='i')
    ax1.set_xticks(range(0, len(input_feature), tick_spacing))
    ax1.set_xticklabels(input_feature[::5], rotation=90)
    ax1.set_xlabel('i')
    ax1.set_ylabel('Error (%)', color='r')
    ax2.set_ylabel('Value', color='b')
    ax1.set_title(name + '_max_error')
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    # min_mean_error 그래프
    ax1 = ax[index, 2]
    ax2 = ax1.twinx()
    ax1.plot(x, i_error[min_mean_error_index], 'r-', label='Error')
    ax2.plot(x, i_original[min_mean_error_index], 'b-', label='Original')
    ax2.plot(x, output[min_mean_error_index], 'g-', label='i')
    ax1.set_xticks(range(0, len(input_feature), tick_spacing))
    ax1.set_xticklabels(input_feature[::5], rotation=90)
    ax1.set_xlabel('i')
    ax1.set_ylabel('Error (%)', color='r')
    ax2.set_ylabel('Value', color='b')
    ax1.set_title(name + '_min_error')
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    # random_index 그래프
    ax1 = ax[index, 3]
    ax2 = ax1.twinx()
    ax1.plot(x, i_error[random_index], 'r-', label='Error')
    ax2.plot(x, i_original[random_index], 'b-', label='Original')
    ax2.plot(x, output[random_index], 'g-', label='i')
    ax1.set_xticks(range(0, len(input_feature), tick_spacing))
    ax1.set_xticklabels(input_feature[::5], rotation=90)
    ax1.set_xlabel('i')
    ax1.set_ylabel('Error (%)', color='r')
    ax2.set_ylabel('Value', color='b')
    ax1.set_title(name + '_random_index')
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

plt.tight_layout()
plt.savefig('error.png')